<a href="https://www.kaggle.com/code/emigiupponi/usdt-ranking-prices-in-argentina?scriptVersionId=161790827" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Analysis of USDT Prices and Spread in ARS: Visualizing Trends and Patterns

In this code, we conduct an analysis of USDT prices in ARS (Argentine Pesos) and daily spread. The process begins by obtaining the current date in the Buenos Aires timezone and creating a CSV file with cryptocurrency platform data. Subsequently, the file is uploaded to an AWS S3 bucket and combined with previous data stored in the same bucket. After calculating statistics and filtering outliers using the interquartile range, we generate an interactive plot with Plotly. The plot showcases the daily average USDT prices, shaded areas indicating standard deviations, and highlights the last days of each month. This visualization provides a clear perspective on the evolution of prices and spread in ARS, aiding in the identification of patterns and trends. Finally, Chart Studio is used to share and visualize the graph more accessibly.

Errors: From Jan 6, 2024, to Feb 4, 2024, we inadvertently collected BTC prices by error.

In [1]:
pip install chart_studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
import requests
import io
import boto3
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from kaggle_secrets import UserSecretsClient
import chart_studio
import chart_studio.plotly as py

In [3]:
# Step 1: Get current date in Buenos Aires timezone
ba_timezone = pytz.timezone('America/Argentina/Buenos_Aires')
ba_datetime = datetime.now(ba_timezone)
date_str = ba_datetime.strftime('%Y-%m-%d')
filename = f'{date_str}_data.csv'

In [4]:
# Step 2: Query data from the API and save to CSV
url = "https://criptoya.com/api/usdt/ars"
json_data = requests.get(url).json()
platform_prices = {
    platform: {
        'ask': data.get('ask', None),
        'totalAsk': data.get('totalAsk', None),
        'bid': data.get('bid', None),
        'totalBid': data.get('totalBid', None),
        'time': data.get('time', None)
    }
    for platform, data in json_data.items()
}
df = pd.DataFrame(platform_prices).T
df.to_csv(filename, index=True)

In [5]:
# Step 3: Upload CSV file to S3
user_secrets = UserSecretsClient()
access_key = user_secrets.get_secret("AWSAccessKeyId")
secret_key = user_secrets.get_secret("AWSSecretKey")
bucket_name = 'usdt-prices-arg'
s3_client = boto3.client('s3', aws_access_key_id=access_key, aws_secret_access_key=secret_key)
existing_buckets = [bucket['Name'] for bucket in s3_client.list_buckets()['Buckets']]
if bucket_name not in existing_buckets:
    s3_client.create_bucket(Bucket=bucket_name)
s3_client.upload_file(filename, bucket_name, filename)

In [6]:
# Step 4: Read and combine CSV files from S3
response = s3_client.list_objects(Bucket=bucket_name)
objects = response.get('Contents', [])

dataframes = []

for obj in objects:
    if obj['Key'].endswith('_data.csv'):
        # Extraer la fecha del nombre del archivo
        file_date = datetime.strptime(obj['Key'][:10], '%Y-%m-%d').date()

        # Leer el archivo CSV y agregar la columna con la fecha
        df = pd.read_csv(io.BytesIO(s3_client.get_object(Bucket=bucket_name, Key=obj['Key'])['Body'].read()))

        # Renombrar columnas según las especificaciones
        df.rename(columns={'Buying': 'ask', 'Buying (final)': 'totalAsk', 'Selling': 'bid', 'Selling (final)': 'totalBid', 'Timestamp': 'time'}, inplace=True)

        # Agregar la columna de fecha
        df['date'] = file_date

        # Agregar el dataframe a la lista
        dataframes.append(df)

# Concatenar todos los dataframes en uno
combined_df = pd.concat(dataframes, ignore_index=True)

In [7]:
# Build new data
combined_df['date'] = pd.to_datetime(combined_df['date'])
combined_df['spread'] = combined_df['totalAsk'] - combined_df['totalBid']
combined_df['average_price'] = (combined_df['totalAsk'] + combined_df['totalBid'])/2

In [8]:
# Calcula el rango intercuartil (IQR) para la columna 'average_price'
Q1 = combined_df['average_price'].quantile(0.25)
Q3 = combined_df['average_price'].quantile(0.75)
IQR = Q3 - Q1

# Define los límites superior e inferior para filtrar
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtra las filas que están dentro del rango intercuartil
filtered_df = combined_df[(combined_df['average_price'] >= lower_bound) & (combined_df['average_price'] <= upper_bound)]

In [9]:
# Step 5: Calculate statistics
grouped_df = filtered_df.groupby('date')
daily_avg_spread = grouped_df['spread'].mean()
daily_avg_ask = grouped_df['totalAsk'].mean()
daily_avg_bid = grouped_df['totalBid'].mean()
daily_avg_price = grouped_df['average_price'].mean()
daily_spread_percentage = daily_avg_spread / daily_avg_price
daily_std_ask = grouped_df['totalAsk'].std()
daily_std_bid = grouped_df['totalBid'].std()
daily_std_spread = daily_avg_spread.std()
daily_std_spread_percentage = daily_spread_percentage.std()

In [10]:
# Step 6: Plot the data
last_day_of_month = daily_avg_price.resample('M').last()
last_day_of_month_spread = daily_spread_percentage.resample('M').last()
last_day_of_month = last_day_of_month[last_day_of_month.index.month != 1]
last_day_of_month_spread = last_day_of_month_spread[last_day_of_month_spread.index.month != 1]
variation_percentage = (last_day_of_month - last_day_of_month.shift(1)) / last_day_of_month.shift(1) * 100
last_day_text = [f'Last Day, {day.strftime("%b %d, %Y")}, {value:.2f} ({var:.2f} MoM %)' for day, value, var in zip(last_day_of_month.index, last_day_of_month.values, variation_percentage)]

fig_times_series = make_subplots(rows=1, cols=2, shared_xaxes=True)

fig_times_series.add_trace(go.Scatter(x=daily_avg_ask.index, y=daily_avg_price.values, name='Daily Average', line=dict(color='#1C598A', width=0.5), marker_color='#1C598A'), row=1, col=1)
fig_times_series.add_trace(go.Scatter(x=daily_avg_ask.index, y=daily_avg_price.values + daily_std_ask, fill='tonexty', mode='none', fillcolor='rgba(102, 169, 224, 0.2)', showlegend=False), row=1, col=1)
fig_times_series.add_trace(go.Scatter(x=daily_avg_ask.index, y=daily_avg_price.values - daily_std_ask, fill='tonexty', mode='none', fillcolor='rgba(102, 169, 224, 0.2)', showlegend=False), row=1, col=1)
fig_times_series.add_trace(go.Scatter(x=last_day_of_month.index, y=last_day_of_month.values, mode='markers', marker=dict(color='#1C598A', size=8), name='Last Day of the Month', text=last_day_text, hoverinfo='text'), row=1, col=1)

fig_times_series.add_trace(go.Scatter(x=daily_spread_percentage.index, y=daily_spread_percentage.values, name='Daily Spread %', line=dict(color='gray', width=0.5), marker_color='gray'), row=1, col=2)
fig_times_series.add_trace(go.Scatter(x=daily_spread_percentage.index, y=daily_spread_percentage.values - daily_std_spread_percentage, fill='tonexty', mode='none', fillcolor='rgba(191, 191, 191, 0.2)', showlegend=False), row=1, col=2)
fig_times_series.add_trace(go.Scatter(x=daily_spread_percentage.index, y=daily_spread_percentage.values + daily_std_spread_percentage, fill='tonexty', mode='none', fillcolor='rgba(191, 191, 191, 0.2)', showlegend=False), row=1, col=2)
fig_times_series.add_trace(go.Scatter(x=last_day_of_month_spread.index, y=last_day_of_month_spread.values, mode='markers', marker=dict(color='gray', size=8), name='Last Day of the Month'), row=1, col=2)

fig_times_series.update_layout(
    title='<b>Average USDT Prices and Spread in ARS<b>',
    title_font_color="#AA322F",
    yaxis=dict(title='ARS', title_standoff=25, title_font=dict(size=18), tickformat='~s'),
    yaxis2=dict(title='% Spread*', title_standoff=25, title_font=dict(size=18), tickformat=',.1%', overlaying='y', side='right'),
    legend=dict(x=0, y=1, traceorder='normal', font=dict(size=12), bgcolor='rgba(0,0,0,0)'),
    margin=dict(l=50, r=50, t=50, b=50),
    showlegend=True
)

fig_times_series.show()

In [11]:
# Step 7: Send to monitor
secret_key = user_secrets.get_secret("CHARTSTUDIO_BCRA_API_KEY")

chart_studio.tools.set_credentials_file(username='crypto-financial-stability-monitor', api_key=secret_key)

# Plot the figure
py.plot(fig_times_series, filename='arg-prices-usdt-times-series', auto_open=True)

'https://plotly.com/~crypto-financial-stability-monitor/743/'